<a href="https://massstreetuniversity.com/"><img src="http://tutorials.massstreetuniversity.com/images/logo.png" alt="School Logo"></a><br><br><br>
<a href="http://tutorials.massstreetuniversity.com/transact-sql/">Tutorial Home</a>

<h1>The Only Bloody Good Reason To Use Cursors</h1>

<h1>(AKA Moving Large Amounts Of Data Between Tables)</h1>

In general, cursors are the devil. They are slow. They are from a processing paradigm of a bygone era. You should be focusing on developing set-based solutions and not loops.

However,  cursors can be useful for loading large amounts of data. I am talking about loads that take hours where, if the load fails, it will take even more hours to unwind. 

In this case, you can use cursors to commit batches of data and create checkpoints in your load process. Below is an example of how to do this.

In [ ]:
--Split a batch up into chunks using a cursor.
--This method can be used for most any large table with some modifications
--It could also be refined further with an @Day variable (for example)
--This is for the case where you already have the data in a table
--and are ready to process it. Obviously, if you have a large flat file,
--you can just use BULK INSERT.

DECLARE @Year INT
DECLARE @Month INT

DECLARE BatchingCursor CURSOR FOR
SELECT DISTINCT YEAR([SomeDateField]),MONTH([SomeDateField])
FROM [Sometable];


OPEN BatchingCursor;
FETCH NEXT FROM BatchingCursor INTO @Year, @Month;
WHILE @@FETCH_STATUS = 0
BEGIN

BEGIN TRANSACTION
--All logic goes in here
--Any select statements from [Sometable] need to be suffixed with:
--WHERE Year([SomeDateField])=@Year AND Month([SomeDateField])=@Month   
COMMIT TRANSACTION

FETCH NEXT FROM BatchingCursor INTO @Year, @Month;
END;
CLOSE BatchingCursor;
DEALLOCATE BatchingCursor;
GO


Copyright © 2020, Mass Street Analytics, LLC. All Rights Reserved.